## csv to json

In [1]:
import csv
import json

# Chemin vers ton fichier CSV
csv_file_path = 'bano-01.csv'
json_file_path = 'bano-01.json'

# Conversion du CSV en JSON
data = []
with open(csv_file_path, mode='r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        data.append({
            "id": row[0],
            "numero": row[1],
            "rue": row[2],
            "code_postal": row[3],
            "ville": row[4],
            "source": row[5],
            "latitude": float(row[6]),
            "longitude": float(row[7])
        })

# Sauvegarde du fichier JSON
with open(json_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)


In [3]:
from elasticsearch import Elasticsearch

# Connexion à Elasticsearch
es = Elasticsearch("http://localhost:9200")

# Nom de l'index à modifier
index_name = "adresses"

# Script pour modifier les données (ajouter un champ geo_point)
update_query = {
    "script": {
        "source": """
            ctx._source.location = ['lat': ctx._source.latitude, 'lon': ctx._source.longitude];
            ctx._source.remove('latitude');
            ctx._source.remove('longitude');
        """,
        "lang": "painless"
    },
    "query": {
        "match_all": {}
    }
}

# Exécuter la requête d'update
response = es.update_by_query(index=index_name, body=update_query)
print(f"{response['updated']} documents modifiés.")


ConnectionTimeout: Connection timed out

## Send data with bulk API

In [2]:
import json
import requests

json_file_path = 'bano-01.json'

with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Configuration de l'URL et des headers pour ElasticSearch
url = "http://localhost:9200/adresses/_bulk"
headers = {'Content-Type': 'application/json'}

bulk_data = ''
for i, doc in enumerate(data):
    # Préparation des données pour l'indexation avec un retour à la ligne
    bulk_data += json.dumps({"index": {"_id": doc["id"]}}) + '\n'
    bulk_data += json.dumps(doc) + '\n'
    
    if i % 500 == 0 and i != 0:
        response = requests.post(url, headers=headers, data=bulk_data)
        if response.status_code != 200:
            print(f"Erreur à l'indexation : {response.text}")
        print(f"Indexed {i} documents")
        bulk_data = ''

if bulk_data:
    response = requests.post(url, headers=headers, data=bulk_data)
    if response.status_code != 200:
        print(f"Erreur à l'indexation : {response.text}")
    print("Indexed remaining documents")


Indexed 500 documents
Indexed 1000 documents
Indexed 1500 documents
Indexed 2000 documents
Indexed 2500 documents
Indexed 3000 documents
Indexed 3500 documents
Indexed 4000 documents
Indexed 4500 documents
Indexed 5000 documents
Indexed 5500 documents
Indexed 6000 documents
Indexed 6500 documents
Indexed 7000 documents
Indexed 7500 documents
Indexed 8000 documents
Indexed 8500 documents
Indexed 9000 documents
Indexed 9500 documents
Indexed 10000 documents
Indexed 10500 documents
Indexed 11000 documents
Indexed 11500 documents
Indexed 12000 documents
Indexed 12500 documents
Indexed 13000 documents
Indexed 13500 documents
Indexed 14000 documents
Indexed 14500 documents
Indexed 15000 documents
Indexed 15500 documents
Indexed 16000 documents
Indexed 16500 documents
Indexed 17000 documents
Indexed 17500 documents
Indexed 18000 documents
Indexed 18500 documents
Indexed 19000 documents
Indexed 19500 documents
Indexed 20000 documents
Indexed 20500 documents
Indexed 21000 documents
Indexed 2150

## Connect Elastic

In [ ]:
#!pip install elasticsearch


In [ ]:
from elasticsearch import Elasticsearch

# Connexion à Elasticsearch
es = Elasticsearch("http://localhost:9200")

# Obtenir la liste de tous les indices
all_indices = es.indices.get(index="*")

# Filtrer pour ne garder que les indices utilisateurs (exclure ceux qui commencent par un point)
user_indices = [index for index in all_indices if not index.startswith('.')]
print("Liste des index utilisateurs :")
for index in user_indices:
    print(index)


In [ ]:
# Afficher le mapping de l'index 'adresses'
mapping = es.indices.get_mapping(index="adresses")
print("Mapping de l'index 'adresses' :")
print(mapping)


In [ ]:
# Obtenir les 10 premiers documents de l'index 'adresses'
response = es.search(index="adresses", body={"query": {"match_all": {}}}, size=10)
documents = response['hits']['hits']
print("Documents dans l'index 'adresses' :")
for doc in documents:
    print(doc['_source'])  # Afficher uniquement le contenu du document


In [ ]:
# Obtenir les coordonnées géographiques
latitudes = [doc['_source']['latitude'] for doc in documents]
longitudes = [doc['_source']['longitude'] for doc in documents]

plt.figure(figsize=(10, 6))
plt.scatter(longitudes, latitudes, alpha=0.5)
plt.title('Distribution des adresses sur la carte')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid()
plt.show()


In [ ]:
# Vérifier le nombre total de documents dans l'index
total_docs = es.count(index="adresses")['count']
print(f"Nombre total de documents dans l'index 'adresses' : {total_docs}")


In [ ]:
import matplotlib.pyplot as plt

# Obtenir les codes postaux
code_postaux_response = es.search(
    index="adresses",
    body={
        "size": 1000,  # Obtenir jusqu'à 1000 documents
        "_source": ["code_postal"]  # Ne récupérer que le champ 'code_postal'
    }
)

code_postaux = [doc['_source']['code_postal'] for doc in code_postaux_response['hits']['hits']]
plt.hist(code_postaux, bins=20)
plt.title('Distribution des codes postaux')
plt.xlabel('Code postal')
plt.ylabel('Fréquence')
plt.show()


## Convert to DF

In [20]:
import pandas as pd
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
response = es.search(index="adresses", body={"query": {"match_all": {}}})
df = pd.DataFrame([doc['_source'] for doc in response['hits']['hits']])


In [ ]:
# Afficher les dernières lignes du DataFrame
df.tail()  # Remplace par df.head() si tu veux voir le début


In [ ]:
df.head()



In [ ]:
# Résumé statistique des colonnes numériques
df.describe()


In [ ]:
# Vérifier les valeurs manquantes dans le DataFrame
missing_values = df.isnull().sum()
print("Valeurs manquantes par colonne :")
print(missing_values)


In [ ]:
#!pip install folium

## Make map 

In [ ]:
# Importer les bibliothèques nécessaires
from elasticsearch import Elasticsearch
import pandas as pd
import folium

# Établir une connexion à Elasticsearch
es = Elasticsearch("http://localhost:9200")

# Vérifier la connexion
if not es.ping():
    print("Erreur de connexion à Elasticsearch!")
else:
    print("Connexion à Elasticsearch réussie!")

# Récupérer toutes les données de l'index 'adresses'
response = es.search(
    index="adresses",
    body={
        "query": {
            "match_all": {}
        },
        "size": 10000  # Ajuster la taille selon le nombre de documents à récupérer
    }
)

# Extraire les documents dans une liste
documents = [doc['_source'] for doc in response['hits']['hits']]

# Créer un DataFrame à partir des documents
df = pd.DataFrame(documents)

# Créer une carte centrée sur la moyenne des coordonnées
map_center = [df['latitude'].mean(), df['longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=13)

# Ajouter les points de données à la carte
for _, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"{row['numero']} {row['rue']}, {row['code_postal']} {row['ville']}",
    ).add_to(m)

# Afficher la carte
m.save("map.html")  # Sauvegarder la carte dans un fichier HTML
m  # Afficher la carte directement dans le notebook


In [9]:
from elasticsearch import Elasticsearch

# Connexion à Elasticsearch
es = Elasticsearch("http://localhost:9200")

def verify_document_structure(index_name, field_definitions):
    """
    Vérifie la structure des documents dans un index Elasticsearch pour s'assurer que chaque champ est conforme à sa définition attendue.
    
    :param index_name: Nom de l'index à vérifier
    :param field_definitions: Dictionnaire des champs attendus et de leur type (ex: {"location": "geo_point"})
    :return: Liste des documents mal formés, nombre total de documents vérifiés
    """
    malformed_docs = []

    # Rechercher tous les documents dans l'index
    response = es.search(index=index_name, body={"query": {"match_all": {}}}, size=10000)
    documents = response['hits']['hits']
    total_docs = len(documents)

    for doc in documents:
        source = doc['_source']
        for field, expected_type in field_definitions.items():
            if field not in source:
                malformed_docs.append({"_id": doc['_id'], "issue": f"Missing field: {field}"})
            elif expected_type == "geo_point":
                # Vérifier que le champ geo_point est bien formé
                if not isinstance(source[field], dict) or 'lat' not in source[field] or 'lon' not in source[field]:
                    malformed_docs.append({"_id": doc['_id'], "issue": f"Malformed geo_point: {source[field]}"})
            elif expected_type == "float":
                # Vérifier que c'est un nombre
                if not isinstance(source[field], (int, float)):
                    malformed_docs.append({"_id": doc['_id'], "issue": f"Field {field} is not a float: {source[field]}"})

    return malformed_docs, total_docs

# Définition des champs attendus et de leurs types
expected_fields = {
    "latitude": "float",
    "longitude": "float",
    "location": "geo_point"
}

# Vérifier la structure des documents dans l'index 'adresses_reformated'
index_name = "test_adresses"
malformed_documents, total_docs = verify_document_structure(index_name, expected_fields)

# Affichage des informations
print(f"Index vérifié : {index_name}")
print(f"Nombre total de documents vérifiés : {total_docs}")

if malformed_documents:
    print(f"Documents mal formés trouvés ({len(malformed_documents)}):")
    for doc in malformed_documents:
        print(doc)
else:
    print("Tous les documents sont correctement formés.")


C:\Users\mehdi.mougin\AppData\Local\Temp\ipykernel_21288\1690141635.py:17: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index=index_name, body={"query": {"match_all": {}}}, size=10000)


NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [test_adresses]', test_adresses, index_or_alias)